In [1]:
%run Functions.py

In [2]:
metricComputed_baseline = 'PupilRelativeBaseline_300ms'
metricComputed_readingStart = 'PupilRelative_readingStarting_300ms'
metricComputed_readingEnd = 'PupilRelative_readingEnding_300ms'
metricComputed_writingStart = 'PupilRelative_writingStarting_300ms'
metricComputed_writingEnd = 'PupilRelative_writingEnding_300ms'
#metricComputed_correlation = 'PupilSizeCorrelation_300ms'


dataFolderName = r'E:\Data\Data' # accessing external hard disk with the data
a = re.compile('(?<=Data\\\\Data\\\\)(.*)(?=\\\\[1-9])')
subjectName_listElement = 3

#dataFolderName = r'C:\DTU\Data\201901_JanuaryExpt' # accessing data saved in the computer
#a = re.compile('(?<=Data\\\\201901_JanuaryExpt\\\\)(.*)(?=\\\\[1-9])')
#subjectName_listElement = 4

resultFileName_baseline = r'C:\DTU\Data\201901_JanuaryExpt\DataExtracted\IndividualTrials\Pupils\Subject_Block_Session_Trial_' \
+ metricComputed_baseline +  '.xlsx'

resultFileName_readingStart = r'C:\DTU\Data\201901_JanuaryExpt\DataExtracted\IndividualTrials\Pupils\Subject_Block_Session_Trial_' \
+ metricComputed_readingStart +  '.xlsx'
resultFileName_readingEnd = r'C:\DTU\Data\201901_JanuaryExpt\DataExtracted\IndividualTrials\Pupils\Subject_Block_Session_Trial_' \
+ metricComputed_readingEnd +  '.xlsx'

resultFileName_writingStart = r'C:\DTU\Data\201901_JanuaryExpt\DataExtracted\IndividualTrials\Pupils\Subject_Block_Session_Trial_' \
+ metricComputed_writingStart +  '.xlsx'
resultFileName_writingEnd = r'C:\DTU\Data\201901_JanuaryExpt\DataExtracted\IndividualTrials\Pupils\Subject_Block_Session_Trial_' \
+ metricComputed_writingEnd +  '.xlsx'

#resultFileName_correlation = r'C:\DTU\Data\201901_JanuaryExpt\DataExtracted\IndividualSessions_TrialAvg\Subject_Block_Session_' \
#+ metricComputed_correlation +  '.xlsx'


for root, dirs, subfolder in os.walk(dataFolderName):
    
    technique = 'dwell_time'
    
    if not dirs:
        
        #if 'notCompleted' in root or 'notInclude' in root: # Some subjects do not have gaze log and have been marked as 
        
        if 'noData' in root or 'Trial' in root or 'trial' in root or 'Nothing' in root: # Some subjects do not have gaze 
            # log and have been marked as notInclude
            continue
        if 'Jonas' in root or 'Praktikant' in root or 'Villads' in root:
            continue
            
        #if 'ac\\3_MS\\' not in root:
        #    continue
        if 'Picture' in root:
            continue
        #if '_MS' not in root:
        #    continue
        
        #if 'bh2\\3\\2019-02-28-16-47-35_1' not in root:
        #    continue
        
        #if '2019-1-16-16-36-17_1stPart_2' not in root:
        #    continue
            
        keysSelected = None
        userKeys = None
        phraseLog = None
        
        for file in subfolder:
            
            
            if fnmatch.fnmatch(file, 'KeySelection*'):
                try:
                    
                    fKeysSelected = open(root + '\\' + file, encoding='utf-8', newline='')
                    readerKeysSelected = csv.reader(fKeysSelected)
                    keysSelected = list(readerKeysSelected)
                    
                    keysSelected.remove(keysSelected[0])
                except:
                    if fKeysSelected is not None:
                        fKeysSelected.close()
                    else:
                        print('error in opening the KeySelection log file')
                        
            
            if fnmatch.fnmatch(file, 'user*'):
                try:
                    fUserKey = open(root + '\\' + file, encoding='utf-8',  newline='')
                    readerUserKey = csv.reader(fUserKey, quotechar=None)
                    userKeys = list(readerUserKey)
                    userKeys.remove(userKeys[0])
                except:
                    if fUserKey is not None:
                        fUserKey.close()
                    else:
                        print('error in opening the user key log file')
                        
            if fnmatch.fnmatch(file, 'phrase*'):
                try:
                    fPhraseLog = open(root + '\\' + file, encoding='utf-8')
                    readerPhraseLog = csv.reader(fPhraseLog, quotechar=None)
                    phraseLog = list(readerPhraseLog)
                    
                except:
                    if fPhraseLog is not None:
                        fPhraseLog.close()
                    else:
                        print('error in opening the phrase log file')
                        
            if fnmatch.fnmatch(file, 'multiKey*'):
                technique = 'multiKey_selection'
            
            if fnmatch.fnmatch(file, 'tobiiGazeLog*'):
                try:
                    fGazeLog = open(root + '\\' + file, encoding='utf-8', newline='')
                    readerGazeLog = csv.reader(fGazeLog, quotechar=None)
                    gazeLog = list(readerGazeLog)
                    gazeLog.remove(gazeLog[0])
                    gazeLog.remove(gazeLog[-1])
                    
                except:
                    if fGazeLog is not None:
                        fGazeLog.close()
                    else:
                        print('error in opening the scratchpad log file')
            
                    
                     
        if keysSelected is None or userKeys is None or phraseLog is None or gazeLog is None:
            continue
        else:
                
            print('subject path', root)
            subjAndSessionName = '__'.join(root.split('\\')[subjectName_listElement:])
            subjName = subjAndSessionName.split('__')[0]
            print('subject and session name: ', subjAndSessionName)
            sessionFolderName = root.split('\\')[-1]
            
            
            # fix phraselog due to comma related file changes
            phraseLog_new = FixScratchPad(phraseLog)
            
            # fix userKeys due to comma related file changes
            userKeys_new = FixUserKeys(userKeys)
            
            
            # need to fix keys selected, where rows get combined because of an inverted comma
            keysSelected_new = FixKeysSelected(keysSelected)
            
            # find start time of typing
            timeTyping = OptiKeyTypingTime(userKeys_new)
            
            
            
            # divide complete data into epochs of phrases
            timeStartEndKeys_reading, timeStartEndKeys_writing = FindExptStartEndTimes(keysSelected_new, timeTyping, root)
            
            userKeys_new_wDwellTime = ComputeDwellTime(userKeys_new, root)
            
            # use the times of trial start and end, to divide the time of trial into reading and writing
            eventTrialsInKeysSelected_reading, eventTrialsInKeysSelected_readingIndex = \
            FindReadingPartsOfTrial_inKeysSelected(timeStartEndKeys_reading, keysSelected_new, root, \
                                                   userKeys_new_wDwellTime)
            eventTrialsInKeysSelected_writing, eventTrialsInKeysSelected_writingIndex = FindWritingPartsOfTrial_inKeysSelected(timeStartEndKeys_writing, keysSelected_new, eventTrialsInKeysSelected_reading)
            
            if sessionFolderName in dict_noGazeData:
                print('no gaze data present')
                pupilSize_baseline = [np.nan]*len(eventTrialsInKeysSelected_writing['start'])
                
                pupilSize_starting_reading = [np.nan]*len(eventTrialsInKeysSelected_writing['start'])
                pupilSize_ending_reading = [np.nan]*len(eventTrialsInKeysSelected_writing['start'])
                pupilSize_starting_writing = [np.nan]*len(eventTrialsInKeysSelected_writing['start'])
                pupilSize_ending_writing = [np.nan]*len(eventTrialsInKeysSelected_writing['start'])
                
            else:
            
                # filter the data
                pupilMean_df, nanValues, correlationRL = FilterPupilSize_wWeightedMean(gazeLog, timeTyping, subjAndSessionName)
            
                # divide trials into reading and writing
                eventReading, eventReading_index = EventPartsFromPupilData(eventTrialsInKeysSelected_reading, pupilMean_df, root)
                eventWriting, eventWriting_index = EventPartsFromPupilData(eventTrialsInKeysSelected_writing, pupilMean_df, root)
                
                
                # baseline time extraction
                #pupilSize_baseLine = GetBaseline(eventTrials_reading, pupilData_filtered)
                timeBaseline = 0.3 # seconds
                pupilSize_baseline = GetBaselineForEveryTrial(eventReading_index, pupilMean_df, nanValues, timeBaseline, \
                                  subjAndSessionName)
                
                #print(pupilSize_baseline)
                # compute the pupil size for the trialComputationTime for reading and writing, at the beginning and end
                # of the parts
                trialComputationTime = 0.3                
                
                #print('reading')
                pupilSize_starting_reading, pupilSize_ending_reading = GetRelativePupilSize_startingAndEnding(\
                                eventReading_index, pupilMean_df, pupilSize_baseline, trialComputationTime, nanValues)
                
                #print('writing')
                pupilSize_starting_writing, pupilSize_ending_writing = GetRelativePupilSize_startingAndEnding(\
                                eventWriting_index, pupilMean_df, pupilSize_baseline, trialComputationTime, nanValues)
                
            
                
            if '1stPart' in root:
                print('1stPart')
                pupilSize_baseline1 = pupilSize_baseline
                pupilSize_starting_reading1 = pupilSize_starting_reading
                pupilSize_ending_reading1 = pupilSize_ending_reading
                pupilSize_starting_writing1 = pupilSize_starting_writing
                pupilSize_ending_writing1 = pupilSize_ending_writing
                
                """
                dataToSave_correlationRL = DataForEverySession()
                dataToSave_correlationRL.subjectID = subjAndSessionName.split('__')[0]
                dataToSave_correlationRL.blockNumber = subjAndSessionName.split('__')[1]
                dataToSave_correlationRL.sessionNumber = subjAndSessionName[-1]
                dataToSave_correlationRL.variable = metricComputed_correlation
                dataToSave_correlationRL.dataForTrial = correlationRL
                dataToSave_correlationRL.resultPathName = resultFileName_correlation
            
                print(dataToSave_correlationRL.printInfo())
                dataToSave_correlationRL.AddToFile()
                """
                continue
            
            
            if '2ndPart' in root:
                print('2ndPart')
                pupilSize_baseline2 = pupilSize_baseline
                
                pupilSize_starting_reading2 = pupilSize_starting_reading
                pupilSize_ending_reading2 = pupilSize_ending_reading
                pupilSize_starting_writing2 = pupilSize_starting_writing
                pupilSize_ending_writing2 = pupilSize_ending_writing
                
                pupilSize_baseline = pupilSize_baseline1 + pupilSize_baseline2
                
                pupilSize_starting_reading = pupilSize_starting_reading1 + pupilSize_starting_reading2
                pupilSize_ending_reading = pupilSize_ending_reading1 + pupilSize_ending_reading2
                pupilSize_starting_writing = pupilSize_starting_writing1 + pupilSize_starting_writing2
                pupilSize_ending_writing = pupilSize_ending_writing1 + pupilSize_ending_writing2
                
                pupilSize_baseline1 = list()
                pupilSize_starting_reading1 = list()
                pupilSize_ending_reading1 = list()
                pupilSize_starting_writing1 = list()
                pupilSize_ending_writing1 = list()
                
                
            print('')
            # save the baseline pupil size for every 
            dataToSave_baseline = DataForEveryTrial()
            dataToSave_baseline.subjectID = subjAndSessionName.split('__')[0]
            dataToSave_baseline.blockNumber = subjAndSessionName.split('__')[1]
            dataToSave_baseline.sessionNumber = subjAndSessionName[-1]
            dataToSave_baseline.variable = metricComputed_baseline
            dataToSave_baseline.dataForTrial = pupilSize_baseline
            dataToSave_baseline.resultPathName = resultFileName_baseline
            
            print(dataToSave_baseline.printInfo())
            
            
            # save the reading start
            dataToSave_readingStart = DataForEveryTrial()
            dataToSave_readingStart.subjectID = subjAndSessionName.split('__')[0]
            dataToSave_readingStart.blockNumber = subjAndSessionName.split('__')[1]
            dataToSave_readingStart.sessionNumber = subjAndSessionName[-1]
            dataToSave_readingStart.variable = metricComputed_readingStart
            dataToSave_readingStart.dataForTrial = pupilSize_starting_reading
            dataToSave_readingStart.resultPathName = resultFileName_readingStart
            
            print(dataToSave_readingStart.printInfo())
            
            
            # save the reading end
            dataToSave_readingEnd = DataForEveryTrial()
            dataToSave_readingEnd.subjectID = subjAndSessionName.split('__')[0]
            dataToSave_readingEnd.blockNumber = subjAndSessionName.split('__')[1]
            dataToSave_readingEnd.sessionNumber = subjAndSessionName[-1]
            dataToSave_readingEnd.variable = metricComputed_readingEnd
            dataToSave_readingEnd.dataForTrial = pupilSize_ending_reading
            dataToSave_readingEnd.resultPathName = resultFileName_readingEnd
            
            print(dataToSave_readingEnd.printInfo())
            
            
            # save the writing start
            dataToSave_writingStart = DataForEveryTrial()
            dataToSave_writingStart.subjectID = subjAndSessionName.split('__')[0]
            dataToSave_writingStart.blockNumber = subjAndSessionName.split('__')[1]
            dataToSave_writingStart.sessionNumber = subjAndSessionName[-1]
            dataToSave_writingStart.variable = metricComputed_writingStart
            dataToSave_writingStart.dataForTrial = pupilSize_starting_writing
            dataToSave_writingStart.resultPathName = resultFileName_writingStart
            
            print(dataToSave_writingStart.printInfo())
            
            
            # save the writing start
            dataToSave_writingEnd = DataForEveryTrial()
            dataToSave_writingEnd.subjectID = subjAndSessionName.split('__')[0]
            dataToSave_writingEnd.blockNumber = subjAndSessionName.split('__')[1]
            dataToSave_writingEnd.sessionNumber = subjAndSessionName[-1]
            dataToSave_writingEnd.variable = metricComputed_writingEnd
            dataToSave_writingEnd.dataForTrial = pupilSize_ending_writing
            dataToSave_writingEnd.resultPathName = resultFileName_writingEnd
            
            print(dataToSave_writingEnd.printInfo())
            
            
        
            """
            dataToSave_baseline.AddToFile()
            dataToSave_readingStart.AddToFile()
            dataToSave_readingEnd.AddToFile()
            dataToSave_writingStart.AddToFile()
            dataToSave_writingEnd.AddToFile()
            
            
            
            
            dataToSave_correlationRL = DataForEverySession()
            dataToSave_correlationRL.subjectID = subjAndSessionName.split('__')[0]
            dataToSave_correlationRL.blockNumber = subjAndSessionName.split('__')[1]
            dataToSave_correlationRL.sessionNumber = subjAndSessionName[-1]
            dataToSave_correlationRL.variable = metricComputed_correlation
            dataToSave_correlationRL.dataForTrial = correlationRL
            dataToSave_correlationRL.resultPathName = resultFileName_correlation
            
            print(dataToSave_correlationRL.printInfo())
            dataToSave_correlationRL.AddToFile()
            """
            
            
            print('')
            
            

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



subject path E:\Data\Data\ac\1\2019-02-11-11-18-30_1
subject and session name:  ac__1__2019-02-11-11-18-30_1
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3325, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-b8a0404c790b>", line 150, in <module>
    timeBaselineQuestion = getBaselineQuestion(timeTyping, keysSelected_new)
NameError: name 'getBaselineQuestion' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2039, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\ul

NameError: name 'getBaselineQuestion' is not defined